# CSP Solver for Wordle game

### Useful links :
- [Medium article](https://medium.com/better-programming/beating-wordle-constraint-programming-ef0b0b6897fe#:~:text=Beating%20Wordle%3A%20Constraint%20Programming,Wordle%20solver%20do%20its%20thing)
- [Sample dataset (GitHub)](https://github.com/dwyl/english-words)


In [1]:
import pandas as pd
from collections import Counter, defaultdict
import itertools

# We import all the english words inside this dataset
total_words = pd.read_fwf("words_alpha.txt", names=["words"])

# Keep only the 5 letter words
words = total_words[total_words["words"].str.len() == 5]

# We convert the words into a list of integers (A -> 0, B -> 1, ..., Z -> 25)
words_data = [tuple([ord(c) - ord('a') for c in word]) for word in words["words"]]

# We initialise the letter statistics for the heuristic function
# Positional frequency (frequency of letters at each position)
positional_freq = [defaultdict(int) for _ in range(5)]
for word in words_data:
    for pos in range(5):
        char = word[pos]
        positional_freq[pos][char] += 1

# Letter frequency (frequency of letters in the dataset)
letter_frequency = Counter(itertools.chain.from_iterable(words_data))

# Normalize the letter frequencies
total_words = len(words_data)
for char, freq in letter_frequency.items():
    letter_frequency[char] = freq / total_words

# Normalize the positional frequencies
for pos in range(5):
    total_pos = sum(positional_freq[pos].values())
    for char, freq in positional_freq[pos].items():
        positional_freq[pos][char] = freq / total_pos

In [10]:
from collections import defaultdict
from ortools.sat.python import cp_model
import random


def get_feedback(guess, target):
    """
    Get the feedback for a given guess and target word and checks duplicate letters
    (G: green, B: black, Y: yellow)
    
    Example:
        guess = "leave"
        target = "place"
        returns : ['Y', 'B', 'G', 'B', 'G']
    """
    feedback = []
    target_counts = Counter(target)
    for g_char, t_char in zip(guess, target):
        if g_char == t_char:
            feedback.append('G')
            target_counts[g_char] -= 1
        else:
            feedback.append('B')

    # Second pass for yellows
    for pos, (g_char, t_char) in enumerate(zip(guess, target)):
        if feedback[pos] == 'B' and g_char in target_counts and target_counts[g_char] > 0:
            feedback[pos] = 'Y'
            target_counts[g_char] -= 1
    return feedback


def update_model(model, position_vars, guess, feedback):
    """
    Update the model using the new feedback
    """
    letter_counts = defaultdict(int)    # Key: letter, Value: count
    gray_positions = defaultdict(list)  # Key: letter, Value: list of positions

    for pos in range(5):
        char = guess[pos]
        fb = feedback[pos]
        if fb == 'G':
            model.Add(position_vars[pos] == char)
            letter_counts[char] += 1
        elif fb == 'Y':
            model.Add(position_vars[pos] != char)
            letter_counts[char] += 1
        elif fb == 'B':
            # Track gray letters for their specific positions
            gray_positions[char].append(pos)

    # Apply gray constraints only to their original positions
    for char, positions in gray_positions.items():
        for pos in positions:
            model.Add(position_vars[pos] != char)

    # Enforce letter counts for Y/G (only if not overridden by grays)
    for char, count in letter_counts.items():
        occurs = [model.NewBoolVar(f'occurs_{p}_{char}') for p in range(5)]
        for p in range(5):
            if p not in gray_positions.get(char, []):  # Skip gray positions
                model.Add(position_vars[p] == char).OnlyEnforceIf(occurs[p])
                model.Add(position_vars[p] != char).OnlyEnforceIf(occurs[p].Not())
        model.Add(sum(occurs) >= count)
    

def update_heuristic(model, position_vars):
    """
    Update the heuristic function to maximize for the model
    """
    # Define the coefficients for the heuristic
    # (FYI: the frequencies are not normalized)
    c_pos_freq = 1000
    c_letter_freq = 2000
    c_dup = 500

    # Build the heuristic objective function
    objective = []
    for pos in range(5):
        char_var = position_vars[pos]
        for char in range(26):
            is_char = model.NewBoolVar(f'pos_{pos}_char_{char}')
            model.Add(char_var == char).OnlyEnforceIf(is_char)
            model.Add(char_var != char).OnlyEnforceIf(is_char.Not())
            
            # Calculate the score based on letter frequencies
            score = int(c_pos_freq * positional_freq[pos][char] + c_letter_freq * letter_frequency[char])
            objective.append(is_char * score)
                
    # Calculate the number of duplicates
    num_duplicates = model.NewIntVar(0, 25, 'num_duplicates')
    char_counts = []
    for char in range(26):
        count = sum([model.NewBoolVar(f'pos_{i}_char_{char}') for i in range(5)])
        for i, pos_var in enumerate(position_vars):
            is_char = model.NewBoolVar(f'pos_{i}_is_char_{char}')
            model.Add(pos_var == char).OnlyEnforceIf(is_char)
            model.Add(pos_var != char).OnlyEnforceIf(is_char.Not())
            count += is_char
        char_counts.append(count)

    # Add constraints to calculate the number of duplicates
    duplicates = []
    for count in char_counts:
        is_duplicate = model.NewBoolVar(f'is_duplicate_{count}')
        model.Add(count > 1).OnlyEnforceIf(is_duplicate)
        model.Add(count <= 1).OnlyEnforceIf(is_duplicate.Not())
        duplicates.append(is_duplicate)

    model.Add(num_duplicates == sum(duplicates))

    # Penalize the objective based on number of duplicates
    objective.append(-c_dup * num_duplicates)

    model.Maximize(sum(objective))


def list_constraints(model):
    """
    Helper function to list all constraints in the model
    """
    model_proto = model.Proto()
    for i, ct in enumerate(model_proto.constraints):
        print(f"Constraint {i}: {ct}")


def filter_valid_words(words_data, guess, feedback):
    """
    Filter all the valid words based on the guess and feedback
    """
    valid = []
    for word in words_data:
        valid_word = True
        # Check green constraints
        for pos in range(5):
            if feedback[pos] == 'G' and word[pos] != guess[pos]:
                valid_word = False
                break
        if not valid_word:
            continue

        # Check yellow constraints
        for pos in range(5):
            if feedback[pos] == 'Y':
                if word[pos] == guess[pos] or guess[pos] not in word:
                    valid_word = False
                    break
        if not valid_word:
            continue
        
        # Check gray constraints
        gray_chars = [guess[pos] for pos in range(5) if feedback[pos] == 'B']
        for char in gray_chars:
            if char in word and char not in [guess[p] for p in range(5) if feedback[p] in ('G', 'Y')]:
                valid_word = False
                break

        if valid_word:
            valid.append(word)
    return valid


def solve_wordle(target_word, max_attempts=6, print_output=True):
    """
    Main function to initialize and run the solver

    Args:
        target_word: the word to solve
        max_attempts: the maximum number of attempts to solve the word
        print_output: whether to print the output of each attempt

    Returns:
        number of tries needed to solve the word (doesn't indicate success)
    """

    # Initialize the model and the valid words
    target_as_int = [ord(c) - ord('a') for c in target_word]
    valid_words = words_data.copy()  # Use a copy to avoid modifying the original

    # Initialize the model and the variables for our model
    model = cp_model.CpModel()
    position_vars = [model.NewIntVar(0, 25, f'pos_{i}') for i in range(5)]
    status_dict = {
        cp_model.OPTIMAL: "OPTIMAL",
        cp_model.FEASIBLE: "FEASIBLE",
        cp_model.INFEASIBLE: "INFEASIBLE",
        cp_model.MODEL_INVALID: "MODEL_INVALID",
        cp_model.UNKNOWN: "UNKNOWN"
    }

    if print_output:    
        print(f"\nPositional frequency: {positional_freq}")
        print(f"Letter frequency: {letter_frequency}\n")
    
    for attempt in range(max_attempts):
        if print_output:
            print(f"Length of possible words: {len(valid_words)}")
            print(f"Is {target_word} in the dataset? {tuple(target_as_int) in valid_words}")

        # Reduce the list of all possible words 
        model.AddAllowedAssignments(position_vars, valid_words)

        # Initialize / update the heuristic function
        update_heuristic(model, position_vars)

        # Initialize and run the solver
        solver = cp_model.CpSolver()
        status = solver.Solve(model)
        if print_output:
            print(f"status = {status_dict.get(status, 'UNKNOWN')}")

        if status == cp_model.OPTIMAL or status == cp_model.FEASIBLE:
            # Extract the guess and feedback
            guess = tuple([solver.Value(pos) for pos in position_vars])
            valid_words.remove(guess)
            guess_str = ''.join([chr(c + ord('a')) for c in guess])
            feedback = get_feedback(guess, target_as_int)
            if print_output:
                print(f"\nAttempt {attempt+1}: {guess_str} → {feedback}")
            
            if feedback == ['G'] * 5:
                print(f"Solved {target_word} in {attempt+1} attempts!")
                return attempt + 1
            
            # Remove invalid words and update the model
            valid_words = filter_valid_words(valid_words, guess, feedback)
            update_model(model, position_vars, guess, feedback)
        
        else:
            print("Model is infeasible. Exiting.")
            return attempt + 1
        
    print(f"Failed to solve {target_word} in {max_attempts} attempts.")
    return max_attempts

# Example test
# for i in range(5):
#     random_word = random.choice(words["words"].to_list())
#     print(f"Random word: {random_word}")
#     solve_wordle(random_word, 100)

random_word = random.choice(words["words"].to_list())
print(f"Random word: {random_word}")
solve_wordle(random_word, 5)


Random word: lindy

Positional frequency: [defaultdict(<class 'int'>, {0: 0.07373908674078261, 1: 0.0716663526160417, 2: 0.07518371961560204, 3: 0.05031091011871114, 4: 0.026443062621694616, 5: 0.04296212549462974, 6: 0.04629106211921362, 7: 0.03586458137051693, 24: 0.010489290873688838, 8: 0.01890584762263677, 9: 0.016330632497958672, 10: 0.029709189121286353, 11: 0.04264807486966899, 12: 0.053325796118334275, 13: 0.025500910746812388, 14: 0.020978581747377677, 15: 0.0592927579925884, 16: 0.005338860624332643, 17: 0.04277369511965329, 18: 0.11387475661076565, 19: 0.06161673261729791, 20: 0.020601720997424786, 21: 0.018026505872746686, 22: 0.029395138496325607, 23: 0.0016958733747880158, 25: 0.007034733999120658}), defaultdict(<class 'int'>, {0: 0.18032786885245902, 1: 0.006846303624144212, 2: 0.015953771748005777, 3: 0.008542176998932227, 4: 0.12379875635952516, 5: 0.0025124049996859492, 6: 0.006406632749199171, 7: 0.04522328999434709, 24: 0.017586834997801646, 8: 0.10483009861189624,

3

### Other implementation, naive version without constraint model
This version only uses the score for each model picks the word with the best score.

It doesn't use google OR-TOOLS and is noticeably faster for almost same results.

In [10]:
import random

# We initialise the word statistics for the solver
# (i.e. the positional frequency of each letter and the global letter frequency)
positional_freq = [defaultdict(int) for _ in range(5)]

for word in words_data:
    for pos in range(5):
        char = word[pos]
        positional_freq[pos][char] += 1

letter_frequency = Counter(itertools.chain.from_iterable(words_data))

# Normalize the letter frequencies
total_words = len(words_data)
for char, freq in letter_frequency.items():
    letter_frequency[char] = freq / total_words

# Normalize the positional frequencies
for pos in range(5):
    total_pos = sum(positional_freq[pos].values())
    for char, freq in positional_freq[pos].items():
        positional_freq[pos][char] = freq / total_pos


print(f"positional_freq: {positional_freq}")
print(f"letter_frequency: {letter_frequency}")

# Precompute scores for all words
coef_pos_freq = 5
coef_let_freq = 1
coef_dup = 1
word_scores = []
for word in words_data:
    score = sum(
        coef_pos_freq * positional_freq[pos][char]
        + coef_let_freq * letter_frequency[char]
        + coef_dup * len(set(word))
        for pos, char in enumerate(word)
    )
    word_scores.append((score, tuple(word)))

# Sort words by descending score
word_scores.sort(reverse=True, key=lambda x: x[0])
sorted_words = [word for (score, word) in word_scores]

def filter_valid_words(words_data, guess, feedback):
    valid = []
    for word in words_data:
        valid_word = True
        # Check green constraints
        for pos in range(5):
            if feedback[pos] == 'G' and word[pos] != guess[pos]:
                valid_word = False
                break
        if not valid_word:
            continue

        # Check yellow constraints
        for pos in range(5):
            if feedback[pos] == 'Y':
                if word[pos] == guess[pos] or guess[pos] not in word:
                    valid_word = False
                    break
        if not valid_word:
            continue

        # Check gray constraints
        gray_chars = [guess[pos] for pos in range(5) if feedback[pos] == 'B']
        for char in gray_chars:
            if char in word and char not in [guess[p] for p in range(5) if feedback[p] in ('G', 'Y')]:
                valid_word = False
                break

        if valid_word:
            valid.append(word)
    return valid


def solve_wordle(target_word, max_attempts=6):
    target_as_int = tuple(ord(c) - ord('a') for c in target_word)
    valid_words = sorted_words.copy()  # Use precomputed sorted list

    for attempt in range(max_attempts):
        if not valid_words:
            print("No valid guesses left!")
            return

        # for i in range(len(valid_words)):
            # print(''.join([chr(c + ord('a')) for c in valid_words[i]]))

        # Pick the highest-scoring valid word
        best_guess = valid_words[0]
        guess_str = ''.join([chr(c + ord('a')) for c in best_guess])
        feedback = get_feedback(best_guess, target_as_int)
        print(f"Attempt {attempt+1}: {guess_str} → {feedback}")

        if feedback == ['G'] * 5:
            print(f"Solved {target_word} in {attempt+1} attempts!")
            return

        # Filter valid words based on feedback
        valid_words = filter_valid_words(valid_words, best_guess, feedback)

    print(f"Failed to solve {target_word} in {max_attempts} attempts.")


# Example usage
random_word = random.choice(words["words"].to_list())
print(f"Random word: {random_word}")
solve_wordle(random_word, 100)


positional_freq: [defaultdict(<class 'int'>, {0: 0.07373908674078261, 1: 0.0716663526160417, 2: 0.07518371961560204, 3: 0.05031091011871114, 4: 0.026443062621694616, 5: 0.04296212549462974, 6: 0.04629106211921362, 7: 0.03586458137051693, 24: 0.010489290873688838, 8: 0.01890584762263677, 9: 0.016330632497958672, 10: 0.029709189121286353, 11: 0.04264807486966899, 12: 0.053325796118334275, 13: 0.025500910746812388, 14: 0.020978581747377677, 15: 0.0592927579925884, 16: 0.005338860624332643, 17: 0.04277369511965329, 18: 0.11387475661076565, 19: 0.06161673261729791, 20: 0.020601720997424786, 21: 0.018026505872746686, 22: 0.029395138496325607, 23: 0.0016958733747880158, 25: 0.007034733999120658}), defaultdict(<class 'int'>, {0: 0.18032786885245902, 1: 0.006846303624144212, 2: 0.015953771748005777, 3: 0.008542176998932227, 4: 0.12379875635952516, 5: 0.0025124049996859492, 6: 0.006406632749199171, 7: 0.04522328999434709, 24: 0.017586834997801646, 8: 0.10483009861189624, 9: 0.0011933923748508259